In [59]:
#import dependencies
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
from splinter import Browser
import pandas as pd
from selenium import webdriver
import json
import pymongo
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, insert, MetaData
import sqlalchemy
import numpy as np
from time import sleep
import requests
from pprint import pprint
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float, Table 


In [60]:
#setup splinter
executable_path = {'executable_path':ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless = False)



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [C:\Users\ravjo\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache


In [ ]:
[WBD] - Current google-chrome version is 97.0.4692
[WBD] - Get LATEST chromedriver version for 97.0.4692 google-chrome
[WBD] - Driver [C:\Users\ravjo\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache

In [ ]:
[WDM] - Driver has been saved in cache [C:\Users\ravjo\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache

In [61]:
engine = create_engine('sqlite:///disney_park.sqlite')
table_drop = engine.execute('DROP TABLE IF EXISTS disney_park')
conn = engine.connect
meta=MetaData()    

    

In [62]:
disney_park = Table(
'disney_park', meta,
Column('index', Integer, primary_key = True),
Column("Disney Park", String(255)),
Column("Park Status", String(255)),)

meta.create_all(engine)

In [63]:
data = engine.execute('select * from disney_park')
session=Session(engine)

In [41]:
# Base = declarative_base()
# class park(Base):
#     __tablename__ = 'disney_park'
#     "index" = Column(Integer, primary_key=True)
#     "Disney Park" = Column(String(255))
#     "Park Status" = Column(String(255))

# engine = create_engine('sqlite:///disney_park.sqlite')
# conn = engine.connect
# session=Session(engine)

# Base.metadata.drop_all(engine)
# Base.metadata.create_all(engine)


# session.commit()

SyntaxError: cannot assign to literal (<ipython-input-41-22cefe425d67>, line 4)

In [64]:
#connect to website
url = 'https://queue-times.com/en-US/parks/17/queue_times'
browser.visit(url)

html = browser.html

#parse HTML with Beautiful Soup
soup = bs(html, 'html.parser')

In [65]:
ride_status_list = []


In [66]:
#gets the current statuses of the each disney park

#pages of each park page
pages = (4,5,6,7,8,16,17,28,30,274,275)

#loop through each page to get the park name and current park status
for page in pages:
    url = 'https://queue-times.com/en-US/parks/' +str(page) + '/queue_times'
    browser.visit(url)
    html = browser.html
    soup = bs(html, 'html.parser')
    park_status = soup.find("p",class_= "subtitle").text.replace('\n',' ')
    status_dict = {}
    ride_status_list.append(status_dict)
   
    
    for i in soup.find_all("div", class_="buttons"):
        
        park = soup.find("h1", class_="title").text.strip('\n')
        park = park.replace('live wait times','')
        v= {'Disney Park':park, 'Park Status':park_status}
        status_dict.update(v)
    
print(status_dict)

    
    

{'Disney Park': 'Tokyo Disney Resort Disney Sea ', 'Park Status': ' Currently open 09:00 - 21:00 '}


In [67]:

print(ride_status_list)


[{'Disney Park': 'Disneyland Park Paris ', 'Park Status': ' Opens in about 2 hours '}, {'Disney Park': 'Epcot ', 'Park Status': ' Opens in about 8 hours '}, {'Disney Park': 'Disney Magic Kingdom ', 'Park Status': ' Opens in about 7 hours '}, {'Disney Park': 'Disney Hollywood Studios ', 'Park Status': ' Opens in about 7 hours '}, {'Disney Park': 'Animal Kingdom ', 'Park Status': ' Opens in about 7 hours '}, {'Disney Park': 'Disneyland ', 'Park Status': ' Opens in about 10 hours '}, {'Disney Park': 'Disney California Adventure ', 'Park Status': ' Opens in about 10 hours '}, {'Disney Park': 'Walt Disney Studios Paris ', 'Park Status': ' Opens in about 2 hours '}, {'Disney Park': 'Shanghai Disney Resort ', 'Park Status': ' Currently open 09:00 - 20:30 '}, {'Disney Park': 'Tokyo Disney Resort Magic Kingdom ', 'Park Status': ' Currently open 09:00 - 21:00 '}, {'Disney Park': 'Tokyo Disney Resort Disney Sea ', 'Park Status': ' Currently open 09:00 - 21:00 '}]


In [68]:
status_df=pd.DataFrame(ride_status_list)
status_df['Disney Park']=status_df['Disney Park'].str.replace('live wait times','')

status_df

,Disney Park,Park Status
0,Disneyland Park Paris,Opens in about 2 hours
1,Epcot,Opens in about 8 hours
2,Disney Magic Kingdom,Opens in about 7 hours
3,Disney Hollywood Studios,Opens in about 7 hours
4,Animal Kingdom,Opens in about 7 hours
5,Disneyland,Opens in about 10 hours
6,Disney California Adventure,Opens in about 10 hours
7,Walt Disney Studios Paris,Opens in about 2 hours
8,Shanghai Disney Resort,Currently open 09:00 - 20:30
9,Tokyo Disney Resort Magic Kingdom,Currently open 09:00 - 21:00


In [69]:

status_df.to_sql('disney_park', con=engine, if_exists="append")


In [70]:
data = engine.execute("SELECT * FROM disney_park")
for row in data:
    print(row.keys)


<bound method Row.keys of (0, 'Disneyland Park Paris ', ' Opens in about 2 hours ')>
<bound method Row.keys of (1, 'Epcot ', ' Opens in about 8 hours ')>
<bound method Row.keys of (2, 'Disney Magic Kingdom ', ' Opens in about 7 hours ')>
<bound method Row.keys of (3, 'Disney Hollywood Studios ', ' Opens in about 7 hours ')>
<bound method Row.keys of (4, 'Animal Kingdom ', ' Opens in about 7 hours ')>
<bound method Row.keys of (5, 'Disneyland ', ' Opens in about 10 hours ')>
<bound method Row.keys of (6, 'Disney California Adventure ', ' Opens in about 10 hours ')>
<bound method Row.keys of (7, 'Walt Disney Studios Paris ', ' Opens in about 2 hours ')>
<bound method Row.keys of (8, 'Shanghai Disney Resort ', ' Currently open 09:00 - 20:30 ')>
<bound method Row.keys of (9, 'Tokyo Disney Resort Magic Kingdom ', ' Currently open 09:00 - 21:00 ')>
<bound method Row.keys of (10, 'Tokyo Disney Resort Disney Sea ', ' Currently open 09:00 - 21:00 ')>


In [102]:
lands_info = []
rides_info =[]


In [103]:
land_pages = ["4","5","6","7","8","16","17", "28", "30"]

#get rides from disney japan that do not exist in a land
path = 'https://queue-times.com/en-US/parks/274/queue_times.json'
ride_json = (requests.get(path))

ride_data = ride_json.json()['rides']
ride_1 = pd.json_normalize(ride_data)
rides_info.append(ride_1)



#get rides from disney parks that exist in lands

for i in land_pages:
    path_3 = 'https://queue-times.com/en-US/parks/' + i + '/queue_times.json'
    response = (requests.get(path_3))
    for x in response:
        land_data = response.json()['lands']
 
        land = pd.json_normalize(land_data,
                              record_path = ['rides'])
    lands_info.append(land)



 

In [104]:
len(lands_info)
lands_df = pd.DataFrame()

lands_df['Disney Park/Resort'] = []
lands_df['Land']= []
lands_df["Ride"]= []
lands_df["Currently Open"] = []
lands_df['Current Wait Time'] = []

In [105]:
jp_rides_no_land=rides_info[0]
jp_rides_no_land = jp_rides_no_land.drop(columns=['id','last_updated'])
jp_rides_no_land= jp_rides_no_land.rename(columns={'name':'Ride',
                              'is_open':'Currently Open',
                              'wait_time':'Wait Time (minutes)'})
jp_rides_no_land["Disney Park"]= "Tokyo Disney Resort Magic Kingdom"
jp_rides_no_land

,Ride,Currently Open,Wait Time (minutes),Disney Park
0,"""it's a small world""",True,10,Tokyo Disney Resort Magic Kingdom
1,【9/28～ OPEN!】Enchanted Tale of Beauty and the ...,True,5,Tokyo Disney Resort Magic Kingdom
2,【9/28～ OPEN!】The Happy Ride with Baymax,True,10,Tokyo Disney Resort Magic Kingdom
3,Alice's Tea Party,True,5,Tokyo Disney Resort Magic Kingdom
4,Beaver Brothers Explorer Canoes,True,5,Tokyo Disney Resort Magic Kingdom
5,Big Thunder Mountain,True,5,Tokyo Disney Resort Magic Kingdom
6,Buzz Lightyear's Astro Blasters,True,10,Tokyo Disney Resort Magic Kingdom
7,Castle Carrousel,True,5,Tokyo Disney Resort Magic Kingdom
8,Chip 'n Dale's Treehouse,False,0,Tokyo Disney Resort Magic Kingdom
9,Cinderella's Fairy Tale Hall,True,10,Tokyo Disney Resort Magic Kingdom


In [87]:
lands_paris=lands_info[0]
lands_paris = lands_paris.drop(columns=['id','last_updated'])
lands_paris = lands_paris.rename(columns={'name':'Ride',
                              'is_open':'Currently Open',
                              'wait_time':'Wait Time (minutes)'})
lands_paris["Disney Park"]= "DisneyLand Park Paris"
lands_paris


,Ride,Currently Open,Wait Time (minutes),Disney Park
0,Adventure Isle,False,0,DisneyLand Park Paris
1,Indiana Jones and the Temple of Peril,False,0,DisneyLand Park Paris
2,Indiana Jones and the Temple of Peril Single R...,False,0,DisneyLand Park Paris
3,La Plage des Pirates,False,0,DisneyLand Park Paris
4,Le Galion des Pirates,False,0,DisneyLand Park Paris
5,Le Passage Enchanté d'Aladdin,False,0,DisneyLand Park Paris
6,Pirates of the Caribbean,False,0,DisneyLand Park Paris
7,Swiss Family Treehouse,False,0,DisneyLand Park Paris
8,Autopia,False,0,DisneyLand Park Paris
9,Buzz Lightyear Laser Blast,False,0,DisneyLand Park Paris


In [86]:
epcot = lands_info[1]

epcot = epcot.rename(columns={'name':'Ride',
                              'is_open':'Currently Open',
                              'wait_time':'Wait Time (minutes)'})
epcot = epcot.drop(columns=['id','last_updated'])
epcot["Disney Park"]= "DisneyWorld Epcot"
epcot

,Ride,Currently Open,Wait Time (minutes),Disney Park
0,Disney & Pixar Short Film Festival,False,0,DisneyWorld Epcot
1,Journey Into Imagination With Figment,False,0,DisneyWorld Epcot
2,Spaceship Earth,False,0,DisneyWorld Epcot
3,Guardians of the Galaxy: Cosmic Rewind,False,0,DisneyWorld Epcot
4,Mission: SPACE,False,0,DisneyWorld Epcot
5,Test Track Presented by Chevrolet,False,0,DisneyWorld Epcot
6,Awesome Planet,False,0,DisneyWorld Epcot
7,Living with the Land,False,0,DisneyWorld Epcot
8,Soarin',False,0,DisneyWorld Epcot
9,The Seas with Nemo & Friends,False,0,DisneyWorld Epcot


In [85]:
dis_world = lands_info[2]
dis_world = dis_world.rename(columns={'name':'Ride',
                              'is_open':'Currently Open',
                              'wait_time':'Wait Time (minutes)'})
dis_world = dis_world.drop(columns=['id','last_updated'])
dis_world["Disney Park"]= "Disney Magic Kingdom"
dis_world


,Ride,Currently Open,Wait Time (minutes),Disney Park
0,A Pirate's Adventure - Treasures of the Seven ...,False,0,Disney Magic Kingdom
1,Jungle Cruise,False,0,Disney Magic Kingdom
2,Pirates of the Caribbean,False,0,Disney Magic Kingdom
3,Swiss Family Treehouse,False,0,Disney Magic Kingdom
4,The Magic Carpets of Aladdin,False,0,Disney Magic Kingdom
5,Walt Disney's Enchanted Tiki Room,False,0,Disney Magic Kingdom
6,Casey Jr. Splash 'N' Soak Station,False,0,Disney Magic Kingdom
7,Cinderella Castle,False,0,Disney Magic Kingdom
8,Dumbo the Flying Elephant,False,0,Disney Magic Kingdom
9,Enchanted Tales with Belle,False,0,Disney Magic Kingdom


In [84]:
hollywood = lands_info[3]
hollywood = hollywood.rename(columns={'name':'Ride',
                              'is_open':'Currently Open',
                              'wait_time':'Wait Time (minutes)'})
hollywood = hollywood.drop(columns=['id','last_updated'])
hollywood["Disney Park"]= "Disney Hollywood Studios"
hollywood


,Ride,Currently Open,Wait Time (minutes),Disney Park
0,Star Wars Launch Bay,False,0,Disney Hollywood Studios
1,Star Wars Launch Bay Theater,False,0,Disney Hollywood Studios
2,Walt Disney Presents,False,0,Disney Hollywood Studios
3,Star Tours® - The Adventures Continue,False,0,Disney Hollywood Studios
4,Vacation Fun - An Original Animated Short with...,False,0,Disney Hollywood Studios
5,Mickey and Minnie's Runaway Railway,False,0,Disney Hollywood Studios
6,Muppet*Vision 3D,False,0,Disney Hollywood Studios
7,Toy Story Midway Mania!,False,0,Disney Hollywood Studios
8,Millennium Falcon: Smugglers Run,False,0,Disney Hollywood Studios
9,Star Wars: Rise of the Resistance,False,0,Disney Hollywood Studios


In [83]:
animal_kg = lands_info[4]
animal_kg = animal_kg.rename(columns={'name':'Ride',
                              'is_open':'Currently Open',
                              'wait_time':'Wait Time (minutes)'})
animal_kg= animal_kg.drop(columns=['id','last_updated'])
animal_kg["Disney Park"]= "Disney Animal Kingdom"
animal_kg


,Ride,Currently Open,Wait Time (minutes),Disney Park
0,Kilimanjaro Safaris,False,0,Disney Animal Kingdom
1,Pangani Forest Exploration Trail,False,0,Disney Animal Kingdom
2,Wildlife Express Train,False,0,Disney Animal Kingdom
3,Expedition Everest - Legend of the Forbidden M...,False,0,Disney Animal Kingdom
4,Kali River Rapids,False,0,Disney Animal Kingdom
5,DINOSAUR,False,0,Disney Animal Kingdom
6,The Boneyard,False,0,Disney Animal Kingdom
7,TriceraTop Spin,False,0,Disney Animal Kingdom
8,Discovery Island Trails,False,0,Disney Animal Kingdom
9,It's Tough to be a Bug!,False,0,Disney Animal Kingdom


In [82]:
cali_dis = lands_info[5]
cali_dis = cali_dis.rename(columns={'name':'Ride',
                              'is_open':'Currently Open',
                              'wait_time':'Wait Time (minutes)'})
cali_dis= cali_dis.drop(columns=['id','last_updated'])
cali_dis["Disney Park"]= "Disneyland"
cali_dis


,Ride,Currently Open,Wait Time (minutes),Disney Park
0,Enchanted Tiki Room,True,5,Disneyland
1,Indiana Jones Adventure,True,15,Disneyland
2,Jungle Cruise,True,5,Disneyland
3,Tarzan's Treehouse,False,0,Disneyland
4,Davy Crockett's Explorer Canoes,False,0,Disneyland
5,Splash Mountain,False,0,Disneyland
6,The Many Adventures of Winnie the Pooh,False,0,Disneyland
7,Alice in Wonderland,True,5,Disneyland
8,Casey Jr. Circus Train,True,5,Disneyland
9,Dumbo the Flying Elephant,True,5,Disneyland


In [106]:
cali_adv = lands_info[6]
cali_adv = cali_adv.rename(columns={'name':'Ride',
                              'is_open':'Currently Open',
                              'wait_time':'Wait Time (minutes)'})
cali_adv= cali_adv.drop(columns=['id','last_updated'])
cali_adv["Disney Park"]= "Disney Califomrnia Adventure"
cali_adv


,Ride,Currently Open,Wait Time (minutes),Disney Park
0,Guardians of the Galaxy – Mission: Breakout!,False,0,Disney Califomrnia Adventure
1,WEB SLINGERS: A Spider-Man Adventure,False,0,Disney Califomrnia Adventure
2,Red Car Trolley - Buena Vista Street,False,0,Disney Califomrnia Adventure
3,Luigi's Rollickin' Roadsters,False,0,Disney Califomrnia Adventure
4,Mater's Junkyard Jamboree,False,0,Disney Califomrnia Adventure
5,Radiator Springs Racers,False,0,Disney Califomrnia Adventure
6,Grizzly River Run,False,0,Disney Califomrnia Adventure
7,Redwood Creek Challenge Trail,False,0,Disney Califomrnia Adventure
8,Soarin' Over California,False,0,Disney Califomrnia Adventure
9,Animation Academy,True,0,Disney Califomrnia Adventure


In [107]:
paris_sds = lands_info[7]
paris_sds = paris_sds.rename(columns={'name':'Ride',
                              'is_open':'Currently Open',
                              'wait_time':'Wait Time (minutes)'})
paris_sds= paris_sds.drop(columns=['id','last_updated'])
paris_sds["Disney Park"]= "Walt Disney Studios Paris"
paris_sds


,Ride,Currently Open,Wait Time (minutes),Disney Park
0,Armageddon : les Effets Spéciaux,False,0,Walt Disney Studios Paris
1,Rock'n'Roller Coaster starring Aerosmith,False,0,Walt Disney Studios Paris
2,Meet Spider Man,False,0,Walt Disney Studios Paris
3,The Twilight Zone Tower of Terror,False,0,Walt Disney Studios Paris
4,Art of Disney Animation,True,0,Walt Disney Studios Paris
5,Cars Quatre Roues Rallye,False,0,Walt Disney Studios Paris
6,Cars ROAD TRIP,False,0,Walt Disney Studios Paris
7,Crush's Coaster,False,0,Walt Disney Studios Paris
8,Crush's Coaster Single Rider,False,0,Walt Disney Studios Paris
9,Flying Carpets Over Agrabah,False,0,Walt Disney Studios Paris


In [108]:
shanghai = lands_info[8]
shanghai  = shanghai.rename(columns={'name':'Ride',
                              'is_open':'Currently Open',
                              'wait_time':'Wait Time (minutes)'})
shanghai = shanghai .drop(columns=['id','last_updated'])
shanghai ["Disney Park"]= "Shanghai Disney Resort"
shanghai 


,Ride,Currently Open,Wait Time (minutes),Disney Park
0,Camp Discovery,True,0,Shanghai Disney Resort
1,Camp Discovery Excavation Site,True,0,Shanghai Disney Resort
2,Challenge Trails at Camp Discovery,True,15,Shanghai Disney Resort
3,Challenge Trails at Camp Discovery,True,15,Shanghai Disney Resort
4,Jungle Friends Happy Circle,True,10,Shanghai Disney Resort
5,Roaring Rapids,True,5,Shanghai Disney Resort
6,Roaring Rapids (Virtual),True,5,Shanghai Disney Resort
7,Selfie Spot with Disney Jungle Characters at H...,False,0,Shanghai Disney Resort
8,Soaring Over the Horizon,True,30,Shanghai Disney Resort
9,Storyhouse Stage,False,0,Shanghai Disney Resort


In [109]:
disney_rides_df=pd.concat([lands_paris,
                          epcot,
                          dis_world,
                          hollywood,
                          animal_kg,
                          cali_dis,
                          cali_adv,
                          paris_sds,
                          shanghai,
                          jp_rides_no_land])


In [110]:
disney_rides_df=disney_rides_df[["Disney Park","Ride","Currently Open", "Wait Time (minutes)"]]
disney_rides_df

,Disney Park,Ride,Currently Open,Wait Time (minutes)
0,DisneyLand Park Paris,Adventure Isle,False,0
1,DisneyLand Park Paris,Indiana Jones and the Temple of Peril,False,0
2,DisneyLand Park Paris,Indiana Jones and the Temple of Peril Single R...,False,0
3,DisneyLand Park Paris,La Plage des Pirates,False,0
4,DisneyLand Park Paris,Le Galion des Pirates,False,0
...,...,...,...,...
42,Tokyo Disney Resort Magic Kingdom,Tokyo Disneyland Electrical Parade Dreamlights,False,0
43,Tokyo Disney Resort Magic Kingdom,Tom Sawyer Island Rafts,True,5
44,Tokyo Disney Resort Magic Kingdom,Toon Park,False,0
45,Tokyo Disney Resort Magic Kingdom,Western River Railroad,True,15


In [111]:

engine_2 = create_engine('sqlite:///disney_ride_wait.sqlite')
table_drop = engine_2.execute('DROP TABLE IF EXISTS disney_ride_wait')
conns = engine_2.connect
meta_2=MetaData()    
disney_ride_wait = Table(
'disney_ride_wait', meta,
Column('index', Integer, primary_key = True),
Column("Disney Park", String(255)),
Column("Ride", String(255)),
Column("Currently Open", String(255)),
Column("Wait Time (minutes)", Float),)


meta.create_all(engine)

In [112]:
disney_rides_df.to_sql('disney_ride_wait', con=engine_2, if_exists="append")
data = engine_2.execute("SELECT * FROM disney_ride_wait")
for row in data:
    print(row.keys)
    print(row)

<bound method Row.keys of (0, 'DisneyLand Park Paris', 'Adventure Isle', 0, 0)>
(0, 'DisneyLand Park Paris', 'Adventure Isle', 0, 0)
<bound method Row.keys of (1, 'DisneyLand Park Paris', 'Indiana Jones and the Temple of Peril', 0, 0)>
(1, 'DisneyLand Park Paris', 'Indiana Jones and the Temple of Peril', 0, 0)
<bound method Row.keys of (2, 'DisneyLand Park Paris', 'Indiana Jones and the Temple of Peril Single Rider', 0, 0)>
(2, 'DisneyLand Park Paris', 'Indiana Jones and the Temple of Peril Single Rider', 0, 0)
<bound method Row.keys of (3, 'DisneyLand Park Paris', 'La Plage des Pirates', 0, 0)>
(3, 'DisneyLand Park Paris', 'La Plage des Pirates', 0, 0)
<bound method Row.keys of (4, 'DisneyLand Park Paris', 'Le Galion des Pirates', 0, 0)>
(4, 'DisneyLand Park Paris', 'Le Galion des Pirates', 0, 0)
<bound method Row.keys of (5, 'DisneyLand Park Paris', "Le Passage Enchanté d'Aladdin", 0, 0)>
(5, 'DisneyLand Park Paris', "Le Passage Enchanté d'Aladdin", 0, 0)
<bound method Row.keys of (6

In [113]:
session.close()